# 6. 递归和优化问题

## 6.1 找零兑换问题
问题描述：自动售货机每次要找给顾客最少数量的硬币。

解法：贪心策略。每次以最多数量的最大面值硬币来迅速减少找零面值。

若币值特殊，贪心策略也可能“失效”！

### 6.1.1 找零问题：递归解法

1. 结束条件：找零金额刚好等于币值。
2. 减小问题规模：对每种硬币尝试一次，选择数量最小的一个。

优点：一定能找到最优解；
缺点：效率低，大量重复计算！

In [5]:
def recMC(coinValueList, change):
    '''
    coinValueList: 币值列表
    change: 找零金额
    '''
    minCoins = change   # 初始化最坏情况：全部用1分硬币
    if change in coinValueList: # 基本结束条件：如果找零金额正好是某个币值
        return 1
    else:   # 递归计算
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recMC(coinValueList, change - i) # 递归调用
            if numCoins < minCoins:
                minCoins = numCoins
    return minCoins

# 调用函数
coinValueList = [1, 5, 10, 25]
change = 63
import time
start = time.time()
print("最少硬币数:", recMC(coinValueList, change))
end = time.time()
print("运行时间:", end - start)

最少硬币数: 6
运行时间: 6.699651002883911


### 6.1.2 找零问题：递归解法改进
关键在于消除重复计算，用一个表将计算过的中间结果保存起来，保存部分找零的最优解。

In [6]:
def recDC(coinValueList, change, knownResults):
    '''
    coinValueList: 币值列表
    change: 找零金额
    knownResults: 已知结果的字典
    '''
    minCoins = change   # 初始化最坏情况：全部用1分硬币
    if change in coinValueList: # 基本结束条件：如果找零金额正好是某个币值
        knownResults[change] = 1    # 记录已知结果
        return 1
    elif change in knownResults: # 查找已知结果
        return knownResults[change]
    else:   # 递归计算
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recDC(coinValueList, change - i, knownResults) # 递归调用
            if numCoins < minCoins:
                minCoins = numCoins
                knownResults[change] = minCoins # 存储已知结果
    return minCoins
start = time.time()
print("最少硬币数（带备忘录）:", recDC([1,5,10,25], 63, {}))
end = time.time()
print("运行时间（带备忘录）:", end - start)

最少硬币数（带备忘录）: 6
运行时间（带备忘录）: 0.0001571178436279297


加上备忘录之后运行速度大幅提升！

### 6.1.3 找零问题：动态规划解法
从一分钱找零的最优解开始，逐步递增，直到最优解。递加过程能保持最优解的关键是，其依赖于更少钱数最优解的简单计算，而更少钱数的最优解已经得到了。

问题的**最优解包括了更小规模子问题的最优解**，这是一个最优化问题能用动态规划策略解决的必要条件。

In [10]:
def dpMakeChange(coinValueList, change, minCoins, coinsUsed):
    '''
    coinValueList: 币值列表
    change: 找零金额
    minCoins: 存储每个金额所需最少硬币数的列表
    coinsUsed: 存储每个金额最后使用的硬币的列表
    '''
    for cents in range(change + 1):
        coinCount = cents   # 初始化最坏情况：全部用1分硬币
        newCoin = 1
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents - j] + 1 < coinCount:
                coinCount = minCoins[cents - j] + 1
                newCoin = j
        minCoins[cents] = coinCount
        coinsUsed[cents] = newCoin
    return minCoins[change]

def printCoins(coinsUsed, change):
    '''
    coinsUsed: 存储每个金额最后使用的硬币的列表
    change: 找零金额
    '''
    coin = change
    while coin > 0:
        thisCoin = coinsUsed[coin]
        print(thisCoin)
        coin = coin - thisCoin

change = 63
coinValueList = [1, 5, 10, 21, 25]
minCoins = [0] * (change + 1)   # 初始化存储最少硬币数的列表
coinsUsed = [0] * (change + 1) # 初始化存储最后使用硬币的列表
print("最少硬币数（动态规划）:", dpMakeChange(coinValueList, change, minCoins, coinsUsed))
printCoins(coinsUsed, change)

最少硬币数（动态规划）: 3
21
21
21


## 6.2 博物馆大盗问题：动态规划
问题描述：在不超过背包容量的前提下，使得宝物价值最大化。

### 定义二进制决策变量
$$
x_i = 
\begin{cases}
1, & \text{选择第 } i \text{ 件宝物} \\
0, & \text{不选择第 } i \text{ 件宝物}
\end{cases}
$$

### 优化问题
$$
\begin{aligned}
\text{最大化} & \quad \sum_{i=1}^{n} v_i x_i \\
\text{约束条件} & \quad \sum_{i=1}^{n} w_i x_i \leq C \\
& \quad x_i \in \{0, 1\}, \quad i = 1, 2, \dots, n
\end{aligned}
$$

### 动态规划递推公式

状态定义：
设 $dp[i][j]$ 表示考虑前 $i$ 件宝物，在背包容量为 $j$ 时的最大价值。

- $i \in \{0, 1, 2, \dots, n\}$：考虑前 $i$ 件宝物
- $j \in \{0, 1, 2, \dots, C\}$：当前可用背包容量
- $dp[i][j]$：在此状态下的最大价值

### 状态转移方程
对于每个 $i > 0$ 和 $j \geq 0$：

$$
dp[i][j] = 
\begin{cases}
dp[i-1][j], & \text{if } j < w_i \quad \text{(放不下第i件宝物)} \\
\max\left(dp[i-1][j], \ dp[i-1][j-w_i] + v_i\right), & \text{if } j \geq w_i \quad \text{(可选择放或不放)}
\end{cases}
$$

### 边界条件
$$
\begin{aligned}
dp[0][j] &= 0, \quad \forall j \in [0, C] \quad \text{(没有宝物可选时价值为0)} \\
dp[i][0] &= 0, \quad \forall i \in [0, n] \quad \text{(背包容量为0时价值为0)}
\end{aligned}
$$

### 最优解
最优总价值为：
$$
V_{\text{max}} = dp[n][C]
$$

### 状态转移解释：
1. **放不下第 $i$ 件宝物** $(j < w_i)$：
   - 只能选择不放入该宝物
   - 价值等于前 $i-1$ 件宝物在容量 $j$ 下的最优解

2. **可以放下第 $i$ 件宝物** $(j \geq w_i)$：
   - 有两种选择：
     - **不放入第 $i$ 件**：价值为 $dp[i-1][j]$
     - **放入第 $i$ 件**：价值为 $dp[i-1][j-w_i] + v_i$
       - $dp[i-1][j-w_i]$：放入当前宝物前的最优解（使用剩余容量）
       - $+ v_i$：加上当前宝物的价值
   - 取两者中的最大值

### 计算顺序：
- 外层循环：$i = 1 \to n$（依次考虑每件宝物）
- 内层循环：$j = 0 \to C$（考虑所有可能的容量）

### 空间优化（滚动数组）

由于 $dp[i][j]$ 只依赖于 $dp[i-1][\cdot]$，可以使用一维数组优化：

```python
dp = [0] * (C + 1)  # 初始化dp数组
for i in range(1, n + 1):
    # 需要逆序遍历，防止重复计算
    for j in range(C, w_i - 1, -1):
        dp[j] = max(dp[j], dp[j - w_i] + v_i)

In [ ]:
# 0-1背包问题动态规划实现
# tr表示物品列表，w表示物品重量，v表示物品价值
tr = [None, {'w':2, 'v':3}, {'w':3, 'v':4}, {'w':4, 'v':5}, {'w':5, 'v':6}]

max_w = 20

m = {(i, w): 0 for i in range(len(tr)) for w in range(max_w + 1)}

for i in range(1, len(tr)):
    for w in range(max_w + 1):
        if tr[i]['w'] > w:
            m[i, w] = m[i - 1, w]
        else:
            m[i, w] = max(m[i - 1, w], m[i - 1, w - tr[i]['w']] + tr[i]['v'])

print(m[len(tr) - 1, max_w])

18


In [ ]:
# 递归+备忘录解法
tr = {(2,3), (3,4), (4,5), (5,6)}
max_w = 20
m = {}

def thief(tr,w):
    if tr == set() or w == 0:
        m[(tuple(tr), w)] = 0
        return 0
    elif (tuple(tr), w) in m:
        return m[(tuple(tr), w)]
    else:
        vmax = 0
        for item in tr:
            if item[0] <= w:
                v = thief(tr - {item}, w - item[0]) + item[1]
                vmax = max(v, vmax)
        m[(tuple(tr), w)] = vmax
        return vmax
print(thief(tr, max_w))

18
